In [4]:
import torch
import torch.nn as nn
import torchtext
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

import os
import pathlib
import random
import string
import re
import numpy as np

#### Download data

In [ ]:
import requests
import io
import zipfile

url = "http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"

response = requests.get(url)

if response.status_code == 200:
    print("Zip file downloaded.")
else:
    print(f"Unable to download zip file: {response.status_code}")
    exit()

zip_content = io.BytesIO(response.content)

with zipfile.ZipFile(zip_content, 'r') as z:
    z.extractall()
    print("Zip file extracted.")

# Get text file
if os.path.isfile("spa-eng/spa.txt"):
    with open("spa-eng/spa.txt", 'r') as f:
        text_lines = f.readlines()

Zip file downloaded.
Zip file extracted.


#### Parsing the data

In [11]:
text_pairs = []
for line in text_lines:
    eng, spa = line.split("\t")
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))

# Sample sentence pairs
for _ in range(5):
    print(random.choice(text_pairs))

('He seems to be a student.', '[start] Parece ser un estudiante.\n [end]')
('The employees are all unionized.', '[start] Todos los empleados están sindicados.\n [end]')
('My orders are absolute.', '[start] Mis órdenes son incondicionales.\n [end]')
("We're historians.", '[start] Somos historiadores.\n [end]')
("That's the reason he became angry.", '[start] Esa es la razón por la que se enfadó.\n [end]')


##### Split the sentence into training set, validation and test set

In [12]:
random.shuffle(text_pairs)

# train_split, val_split, test_split = 0.15
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


##### Vectorization

In [14]:
strip_chars = string.punctuation + "¿"
print(strip_chars)
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")
print(strip_chars)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~¿
!"#$%&'()*+,-./:;<=>?@\^_`{|}~¿


In [ ]:
vocab_size = 15000
sequence_length = 20
batch_size = 64


def custom_standardization(input_strin):
    lowercase = torchtext.transforms


eng_vectorization = 